In [2]:
%pip install ifcopenshell
%pip install lark numpy ## some package missing in the ifcopenshell

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import ifcopenshell
model = ifcopenshell.open('./AC20-FZK-Haus.ifc')

In [4]:
print(model.schema)

IFC4


In [5]:
walls = model.by_type('IfcWall')
print(walls[0])

#15042=IfcWallStandardCase('2XPyKWY018sA1ygZKgQPtU',#12,'Wand-Int-ERDG-4',$,$,#14983,#15037,'BC6F0F70-6195-495E-A2-FC-239713029DB1',$)


In [6]:

import ifcopenshell.util.selector
wall_elements = ifcopenshell.util.selector.filter_elements(model, "IfcWall, IfcSlab")


In [7]:
# Function to get material names
def get_materials(element):
    materials = []
    for rel in element.HasAssociations:
        if rel.is_a("IfcRelAssociatesMaterial"):
            material = rel.RelatingMaterial
            if material.is_a("IfcMaterial"):
                materials.append(material.Name)
            elif material.is_a("IfcMaterialList"):
                materials += [mat.Name for mat in material.Materials]
            elif material.is_a("IfcMaterialLayerSetUsage"):
                materials.append(material.ForLayerSet.MaterialLayers[0].Material.Name)
    return materials

# Function to get dimensions (simplified example, actual extraction might be more complex)
def get_dimensions(element):
    # This is a simplified example. You'd need to adapt this logic based on the element type
    # and how dimensions are stored in your specific IFC files.
    if element.is_a("IfcWallStandardCase"):
        # Extract dimensions for a standard wall as an example
        length = element.OverallWidth
        height = element.OverallHeight
        return {"length": length, "height": height}
    return {}

In [8]:
material_thickness_list = []
for wall in wall_elements:
    materials = get_materials(wall)
    
    print(f"Element ID: {wall.GlobalId}, Type: {wall.is_a()}, Materials: {materials}")

Element ID: 25fsbPyk15VvuXI$yNKenK, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 16DNNqzfP2thtfaOflvsKA, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 1bzfVsJqn8De5PukCrqylz, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 25OWQvmXj5BPgyergP43tY, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 2XPyKWY018sA1ygZKgQPtU, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 3rPX_Juz59peXXY6wDJl18, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 1pPHnf7cXCpPsNEnQf8_6B, Type: IfcSlab, Materials: ['Stahlbeton 65690']
Element ID: 3PfS__Y_DBAfq5naM6zD2Z, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 0knNIAVBPBFvBy_m5QVHsU, Type: IfcWallStandardCase, Materials: ['Leichtbeton 102890359']
Element ID: 07Enbsqm9C7AQC9iyBwfSD, Type: IfcSlab, Materials: ['Solid 397409098']
Element ID: 1$wmdwWPjDYuku_ghVkynE,

In [11]:
## Getting all materials
def find_materials(ifc_file):
    materials = {}
    for material in ifc_file.by_type("IfcMaterial"):
        materials[material.id()] = {"name": material.Name, "used_in": []}
    return materials



{14521: {'name': 'Holz', 'used_in': []}, 15046: {'name': 'Leichtbeton 102890359', 'used_in': []}, 20378: {'name': 'Radial Gradient Fill 1515460218', 'used_in': []}, 34513: {'name': 'Stahlbeton 65690', 'used_in': []}, 35176: {'name': 'Solid', 'used_in': []}, 59294: {'name': 'Solid 397409098', 'used_in': []}}


In [14]:
# Function to find where materials are used
def find_material_usage(ifc_file, materials):
    for relAssociatesMaterial in ifc_file.by_type("IfcRelAssociatesMaterial"):
        material_definition = relAssociatesMaterial.RelatingMaterial
        if material_definition.is_a("IfcMaterial"):
            material_id = material_definition.id()
            if material_id in materials:
                for relatedObject in relAssociatesMaterial.RelatedObjects:
                    materials[material_id]["used_in"].append((relatedObject.is_a(), relatedObject.GlobalId))
        elif material_definition.is_a("IfcMaterialList") or material_definition.is_a("IfcMaterialLayerSetUsage"):
            # Handling for material lists and layer sets if needed
            pass
    return materials

In [15]:
materials = find_materials(model)

# Find where each material is used
materials = find_material_usage(model, materials)

print(materials)


{14521: {'name': 'Holz', 'used_in': [('IfcStair', '38a9vdh9bF5Qg28GWyHhlr')]}, 15046: {'name': 'Leichtbeton 102890359', 'used_in': []}, 20378: {'name': 'Radial Gradient Fill 1515460218', 'used_in': [('IfcBeam', '3tCgZT92j6fw8fXgwCL3Jm')]}, 34513: {'name': 'Stahlbeton 65690', 'used_in': []}, 35176: {'name': 'Solid', 'used_in': [('IfcMember', '0oTQ6V1VbChulreA_hfmUa'), ('IfcMember', '1tpUOcVEPCGgFqSz0VbniQ'), ('IfcMember', '1ulHlNTuj798Os3k9phhVC'), ('IfcMember', '1OckORehXEoB_KILQGJao5'), ('IfcMember', '0nsN_jkIrBQBuGFMg_zMEg'), ('IfcMember', '31Quw2HAvFqv$9_JgCuemp'), ('IfcMember', '35W3cFc4L7MhJR0gIk9uis'), ('IfcMember', '3F58AqxJfF$QkuoceQhWg3'), ('IfcMember', '1nGDFBBHvBJ9NmlweQO0fX'), ('IfcMember', '3qFc3Wrr9BBPJyb2wB$4Ra'), ('IfcMember', '37ab3q_iz8XRterfnMKTQX'), ('IfcMember', '3xdLwxTMD7bw4hWoOQL41M'), ('IfcMember', '2OHMT7B69F5PRG_f3rMcjF'), ('IfcMember', '34zPS2PUTBDOe7g7RIRXK0'), ('IfcMember', '0VzptM87L6uOjQj08tTXIQ'), ('IfcMember', '2CxE0$jXr8iOH1VC2gLCXX'), ('IfcMember', '